In [1]:
###### import numpy as np
from data_iterator import DataIterator,prepare_data
import tensorflow as tf
# from model import *
import time
import random
import sys
# from utils import *
import tensorflow as tf
import tensorflow.keras.layers as layer
# import tensorflow.


In [2]:
EMBEDDING_DIM = 18
HIDDEN_SIZE = 18 * 2
ATTENTION_SIZE = 18 * 2
best_auc = 0.0


In [3]:
train_file = "./data/local_train_splitByUser"
test_file = "./data/local_test_splitByUser"
uid_voc = "./data/uid_voc.pkl"
mid_voc = "./data/mid_voc.pkl"
cat_voc = "./data/cat_voc.pkl"
batch_size = 128
maxlen = 100
test_iter = 100
save_iter = 100
model_type = 'DNN'
seed = 2

In [4]:
model_path = "dnn_save_path/ckpt_noshuff" + model_type + str(seed)
best_model_path = "dnn_best_model/ckpt_noshuff" + model_type + str(seed)
#     gpu_options = tf.GPUOptions(allow_growth=True)
train_data = DataIterator(train_file, uid_voc, mid_voc, cat_voc, batch_size, maxlen, shuffle_each_epoch=False)
test_data = DataIterator(test_file, uid_voc, mid_voc, cat_voc, batch_size, maxlen)
n_uid, n_mid, n_cat = train_data.get_n()    

In [25]:
n_uid

543060

# 输入数据
[uid, mid, cat, mid_list, cat_list, noclk_mid_list, noclk_cat_list]

[float(ss[0]), 1-float(ss[0])]

# 模型

In [5]:
class din_attention(tf.keras.layers.Layer):
    def __init__(self):
        super(din_attention, self).__init__()
        self.fc_part = fc_layer([80,40,1])
        
    def call(self, item_emb, hist_item_emb,item_mask,softmax_stag=True, mode='SUM'):
        """
        item_emb: (B,D)
        hist_item_emb: (B,T,D)
        item_mask: (B,T)
        此处的D均表示cat_embedding维度+item_embedding维度
        """
        masks = tf.math.equal(item_mask, tf.ones_like(item_mask)) # (B,T)
        item_size = item_emb.shape[-1]  
        item_hist_size = hist_item_emb.shape[-1] 
        # 为了方便计算相似度，将item复制至个数与hist_item长度相同
        queries = tf.tile(item_emb, [1,hist_item_emb.shape[1]]) #(B, D *T)
        queries = tf.reshape(queries, [-1,hist_item_emb.shape[1],hist_item_emb.shape[2]])  # (B,T,D)
        din_all = tf.concat([queries, hist_item_emb,queries-hist_item_emb, queries*hist_item_emb],axis=-1)  #(B,T,D*4)
        # 相似度得分
        similar_scores = self.fc_part(din_all) # (B,T,1)
        similar_scores = tf.reshape(similar_scores, [-1,1,hist_item_emb.shape[1]])  # (B,1,T)
#         print(din_all.shape,'scores',similar_scores.shape)
        
        # tf.where 根据mask将得分与hist_item_emb对应的位置匹配填充，不存在的地方，填充为极大值。
        key_masks = tf.expand_dims(masks, 1) # (B, 1, T)
        # 计分器容器
        paddings = tf.ones_like(similar_scores) * (-2 ** 32 + 1)
        
        scores = tf.where(key_masks, similar_scores, paddings) # (B, 1, T)
        
        if softmax_stag:
            scores = tf.nn.softmax(scores) # (B,1,T)
        # weighted sum
        
        output = tf.matmul(scores, hist_item_emb)  # (B,1,T)*(B,T,D) = (B, 1, D)
        output = tf.squeeze(output)     # (B, D)
        return output

In [6]:
class dice_layer(layer.Layer):
    def __init__(self,axis=-1, epsilon = 1e-10):
#         super(dice_layer, self).__init__()
        super().__init__()
        self.axis = axis
        self.epsilon = epsilon
        
    def build(self, input_shape):
        rand = tf.random_normal_initializer()(shape=[input_shape[-1]])
        self.alpha = tf.Variable(rand, dtype=tf.float32, name="alpha")
    def call(self, x):
        input_shape = list(x.shape)
        
        # 需要进行reduce计算的轴
        reducetion_axes = list(range(len(input_shape)))  # [0,1]
        del reducetion_axes[self.axis]      # [0]
        
        broadcast_shape = [1] * len(input_shape)
        broadcast_shape[self.axis] = input_shape[self.axis]  # [1,72]
        # 计算每个样本的某个特征f的均值        
        mean = tf.reduce_mean(x, axis= reducetion_axes) #(f,) 一共f个特征
        brodcast_mean = tf.reshape(mean, broadcast_shape) # reshape (1,f)
        
        # 计算方差
        std = tf.reduce_mean(tf.square(x-brodcast_mean) + self.epsilon, axis=reducetion_axes)
        std = tf.sqrt(std)
        
        # 还原
        brodcast_std = tf.reshape(std,broadcast_shape)
        
        # 标准化，_x的shape不变
        x_normal = (x-brodcast_mean)/ (brodcast_std + self.epsilon)
        
        # 以上过程即为BatchNormalization：
        # x_normed = layer.BatchNormalization(center=False, scale=False)(_x)
        
        # 标准化后使用 sigmoid 函数得到 x_p
        x_p = tf.sigmoid(x_normal)
        return self.alpha * (1.0 - x_p) * x + x_p * x
        
        

In [7]:
class fc_layer(tf.keras.layers.Layer):
    def __init__(self, fc_config=[128,64,1]):
        super(fc_layer, self).__init__()
        self.fc_layers = []
        self.dice_layers = []
       
        
        for s in fc_config[:-1]:
            self.fc_layers.append(layer.Dense(s, name='dense'+str(s)))
            
        for s in fc_config[:-1]:
            self.dice_layers.append(dice_layer())

        self.fc_layers.append(layer.Dense( fc_config[-1]))
        
    def call(self, inp):
        fc_out = inp
        for i, _ in enumerate(self.fc_layers[:-1]):
                fc_out = self.dice_layers[i](self.fc_layers[i](fc_out))
        fc_out =  self.fc_layers[-1](fc_out)       
        return fc_out
#         self.y_hat = tf.nn.softmax(dnn3) + 0.00000001

In [8]:
class DIN(tf.keras.Model):
    def __init__(self, user_count, item_count, cate_count, user_dim, item_dim, cate_dim,EMBEDDING_DIM, HIDDEN_SIZE, ATTENTION_SIZE, use_negsampling = False):
        super(DIN,self).__init__()
        # 初始化用户、商品、类别 Embedding. l2正则化
        self.user_emb= layer.Embedding(user_count,user_dim,
                                              embeddings_regularizer=tf.keras.regularizers.l2(0.1),name='uid_embedding')
        self.item_emb = layer.Embedding(item_count,item_dim,
                                              embeddings_regularizer=tf.keras.regularizers.l2(0.1),name='item_embedding')
        self.cate_emb = layer.Embedding(cate_count,cate_dim,
                                              embeddings_regularizer=tf.keras.regularizers.l2(0.1),name='cat_embedding')
#         # 给定一个商品的偏置项
#         self.item_bisa = tf.Variable(tf.zeros([item_count]), trainable = True)
        
         # 构造模型的全连接层
        self.fc = fc_layer([128,60,2])
        # attention层
        self.din_att = din_attention()
        
        
    def get_emb(self,usr,item,cat,item_his, cat_his):
        user_emb = self.user_emb(usr)
        item_emb = self.item_emb(item)
        cat_emb = self.cate_emb(cat)
        item_his_emb = self.item_emb(item_his)
        cat_his_emb = self.item_emb(cat_his)
        # 合并 concat
        new_item_emb = tf.concat([item_emb,cat_emb], axis=-1)
        new_item_his_emb = tf.concat([item_his_emb, cat_his_emb], axis=-1)
        
        return user_emb,new_item_emb,new_item_his_emb
    
    def call(self,usr,item,cat,item_his, cat_his,item_mask,sl):
        
        user_emb, item_emb, hist_item_emb = self.get_emb(usr, item,cat, item_his,cat_his)
        hist_item_emb_sum = tf.reduce_sum(hist_item_emb,axis=1)
#         print(hist_item_emb)
        att_feat = self.din_att(item_emb, hist_item_emb,item_mask)
        
        inp = tf.concat([user_emb,item_emb,item_emb*hist_item_emb_sum,hist_item_emb_sum,att_feat], axis=-1) # (B,D+4*2D) = (B,9D)(inp)
        out = self.fc(inp)
        y_hat = tf.nn.softmax(out) +  0.00000001
#         print(y_hat)
        return y_hat
    
# (uids, mids, cats, mid_his, cat_his, mid_mask, sl)

In [9]:
din_model = DIN(n_uid, n_mid, n_cat,8,8,8,EMBEDDING_DIM, HIDDEN_SIZE,ATTENTION_SIZE)
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
i = 0  
for src, tgt in train_data:
    i += 1
    uids, mids, cats, mid_his, cat_his, mid_mask, target, sl, noclk_mids, noclk_cats = prepare_data(src, tgt, maxlen, return_neg=True)
    with tf.GradientTape() as tape:
        y_pred = din_model(uids, mids, cats, mid_his, cat_his, mid_mask, sl)
        # loss = -tf.math.reduce_mean(tf.math.log(y_pred)*target)
        
        loss = tf.keras.losses.categorical_crossentropy(target, y_pred)
        loss = tf.reduce_mean(loss)
        # print('batch %d: loss %f'%(i, loss.numpy()))
        if i % 100 == 0:
            print('batch %d: loss %f'%(i, loss.numpy()))
    grads = tape.gradient(loss, din_model.variables)
    optimizer.apply_gradients(grads_and_vars=zip(grads, din_model.variables))


batch 100: loss 0.690731
batch 200: loss 0.677909
batch 300: loss 0.682946
batch 400: loss 0.693110
batch 500: loss 0.680880
batch 600: loss 0.655296
batch 700: loss 0.676379
batch 800: loss 0.671567
batch 900: loss 0.676579


KeyboardInterrupt: 

In [32]:
din_model.variables

[<tf.Variable 'din_8/uid_embedding/embeddings:0' shape=(543060, 8) dtype=float32, numpy=
 array([[-0.0244068 , -0.02550985, -0.0203084 , ...,  0.03258035,
         -0.0292063 , -0.03122671],
        [ 0.03883983, -0.04837803,  0.02451203, ..., -0.00319593,
         -0.0217443 , -0.03688271],
        [ 0.03026816, -0.01825268, -0.03193154, ...,  0.0018046 ,
         -0.00199163,  0.03144531],
        ...,
        [ 0.02640063,  0.01750937,  0.00057988, ...,  0.03822589,
          0.04432417,  0.01043584],
        [ 0.01310742, -0.01339521, -0.03944961, ...,  0.04612417,
          0.04859703, -0.01004821],
        [-0.03743891, -0.03023068, -0.0460321 , ..., -0.02367139,
          0.01235598,  0.00025745]], dtype=float32)>,
 <tf.Variable 'din_8/item_embedding/embeddings:0' shape=(367983, 8) dtype=float32, numpy=
 array([[-0.0150794 ,  0.0161626 , -0.02831387, ..., -0.042497  ,
          0.00786065,  0.04665382],
        [ 0.02042398, -0.02946409, -0.0442545 , ..., -0.00830471,
          

In [31]:
din_model.summary()

Model: "din_8"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
uid_embedding (Embedding)    multiple                  4344480   
_________________________________________________________________
item_embedding (Embedding)   multiple                  2943864   
_________________________________________________________________
cat_embedding (Embedding)    multiple                  12808     
_________________________________________________________________
fc_layer_16 (fc_layer)       multiple                  17394     
_________________________________________________________________
din_attention_8 (din_attenti multiple                  8601      
Total params: 7,695,130
Trainable params: 7,695,130
Non-trainable params: 0
_________________________________________________________________


In [20]:
# din_model = DIN(n_uid, n_mid, n_cat,8,8,8,EMBEDDING_DIM, HIDDEN_SIZE,ATTENTION_SIZE)
# optimizer = tf.keras.optimizers.SGD(learning_rate=5e-4)
# for itr in range(3):
#     i = 0  
#     for src, tgt in train_data:
#         i += 1
#         uids, mids, cats, mid_his, cat_his, mid_mask, target, sl, noclk_mids, noclk_cats = prepare_data(src, tgt, maxlen, return_neg=True)
#         with tf.GradientTape() as tape:
#             y_pred = din_model(uids, mids, cats, mid_his, cat_his, mid_mask, sl)
#             loss = -tf.math.reduce_mean(tf.math.log(y_pred)*target)
#             if i % 100 == 0:
#                 print('batch %d: loss %f'%(i, loss.numpy()))
#         grads = tape.gradient(loss, din_model.variables)
#         optimizer.apply_gradients(grads_and_vars=zip(grads, din_model.variables))
            

SyntaxError: invalid syntax (<ipython-input-20-94da206ad393>, line 11)

In [222]:
# target

[3, 2, 1]

In [ ]:
class Solution:
    def permutation(self, s: str) -> List[str]:
        c, res = list(s), []
        # 循环递归
        # 递归函数
        def dfs(x):
            # 递归退出
            if x == len(c):
                res.append(''.join(c))
                return

            # 标志是否重复的容器
            dic = set()
            for i in range(x, len(c)):
                if c[i] in dic:continue # 重复，因此剪枝
                dic.add(c[i])
                # 交换
                c[i], c[x] = c[x], c[i]
                # 递归
                dfs(x+1)
                # 恢复交换
                c[x], c[i] = c[i], c[x]
        dfs(0)
        return res